# ACO Hyperparameter Tuning for Graph Coloring Problem

This notebook performs hyperparameter tuning for Ant Colony Optimization (ACO) algorithm applied to the Graph Coloring Problem using Optuna.

## 1. Environment Setup

In [ ]:
import sys
import os

# Check if running in Google Colab environment
IS_COLAB = 'google.colab' in sys.modules

print(f"Running in Google Colab: {IS_COLAB}")

if IS_COLAB:
    print("Colab environment detected. Will mount Google Drive.")
    # Mount Google Drive if running in Colab
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully at /content/drive")
else:
    print("Local environment detected. Using local paths.")

In [ ]:
# Configure paths for data, studies, results, and figures based on the execution environment.
from pathlib import Path

# Configure base paths based on environment
if IS_COLAB:
    # Update this path to match your Google Drive structure
    BASE_PATH = Path('/content/drive/MyDrive/meta_graph_coloring_antcol/assignemnt3')
    CODE_PATH = BASE_PATH / 'code'
    # Add code path to system path for imports
    sys.path.insert(0, str(CODE_PATH))
else:
    # Local environment paths
    BASE_PATH = Path('/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3')
    CODE_PATH = BASE_PATH / 'code'

# Define data root path (contains tiny_dataset and main_dataset)
DATA_ROOT = BASE_PATH / 'data'

# Verify paths exist
if not BASE_PATH.exists():
    raise FileNotFoundError(f"Base path does not exist: {BASE_PATH}")
if not DATA_ROOT.exists():
    raise FileNotFoundError(f"Data root does not exist: {DATA_ROOT}")

print(f"Base Path: {BASE_PATH}")
print(f"Code Path: {CODE_PATH}")
print(f"Data Root: {DATA_ROOT}")
print(f"\nPath verification: OK")

In [ ]:
# Install required packages if running in Colab
if IS_COLAB:
    print("Installing required packages...")
    !pip install -q networkx==3.2.1 matplotlib==3.8.2 pandas==2.1.4 numpy==1.26.2 optuna==3.5.0 scikit-learn==1.4.0 scipy==1.11.4
    print("Packages installed successfully!")

In [ ]:
# Import Required Libraries
from pathlib import Path
import multiprocessing
from datetime import datetime
from IPython.display import Image, display, Markdown, HTML

# Import project modules
from dataloader import GraphDataLoader
from optuna_tuner import OptunaACOTuner
from aco_gpc import ACOGraphColoring
from objective_function import aco_objective_function

print("All libraries imported successfully!")

## 2. Study Configuration

Define dataset selection, study name, and hyperparameter search space.

In [ ]:
# Dataset selection: 'tiny_dataset' for quick testing, 'main_dataset' for full experiments
DATASET_NAME = 'main_dataset'  # Change to 'main_dataset' for full tuning

# Study continuation mode
# Option 1: Continue existing study - set CONTINUE_STUDY to the study name
# Example: CONTINUE_STUDY = 'aco_study_tiny_dataset_20251129_172927'
# Option 2: Start new study - set CONTINUE_STUDY = None
CONTINUE_STUDY = None  # Set to study name to continue, or None for new study

# Study name (used for new studies or when CONTINUE_STUDY is None)
if CONTINUE_STUDY:
    STUDY_NAME = CONTINUE_STUDY
    print(f"Continuing existing study: {STUDY_NAME}")
else:
    STUDY_NAME = f'aco_study_{DATASET_NAME}_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
    print(f"Creating new study: {STUDY_NAME}")

# Number of Optuna trials for hyperparameter tuning
N_TRIALS = 100

# Number of random exploration trials before optimization starts
N_STARTUP_TRIALS = 5

# ACO verbose setting 
ACO_VERBOSE = False   # Set to True to see detailed ACO progress

# Hyperparameter search space configuration (parameters to optimize)
PARAM_CONFIG = {
    'iterations': {
        'type': 'int',
        'low': 10,
        'high': 200,
    },
    'alpha': {
        'type': 'float',
        'low': 0.5,
        'high': 3.0,
    },
    'beta': {
        'type': 'float',
        'low': 1.0,
        'high': 6.0,
    },
    'rho': {
        'type': 'float',
        'low': 0.001,
        'high': 0.5,
    },
    'ant_count': {
        'type': 'int',
        'low': 10,
        'high': 50,
    },
    'Q': {
        'type': 'float',
        'low': 0.1,
        'high': 10.0,
    },
    'patience': {
        'type': 'float',
        'low': 0.1,
        'high': 0.3,
    }
}

print(f"Configuration:")
print(f"  Dataset: {DATASET_NAME}")
print(f"  Study Name: {STUDY_NAME}")
print(f"  Optuna Trials: {N_TRIALS}")
print(f"  Random Exploration Trials: {N_STARTUP_TRIALS} (before optimization)")
print(f"\nHyperparameters to Optimize:")
for param_name, param_spec in PARAM_CONFIG.items():
    print(f"  {param_name}: [{param_spec['low']}, {param_spec['high']}] ({param_spec['type']})")

In [ ]:
# Initialize the Optuna tuner
tuner = OptunaACOTuner(
    study_name=STUDY_NAME,
    data_root=str(DATA_ROOT),
    direction='minimize'  # We want to minimize the number of colors used
)

print(f"Tuner initialized with study: {STUDY_NAME}")
print(f"Data root: {DATA_ROOT}")

# Load tuning dataset once (before optimization)
print("\nLoading tuning dataset...")
data_loader = GraphDataLoader(str(DATA_ROOT), DATASET_NAME)
tuning_graphs = data_loader.load_tuning_dataset()
print(f"Loaded {len(tuning_graphs)} graphs for tuning\n")

# Wrapper function to pass tuning graphs to objective function
def objective_wrapper(trial, params, **kwargs):
    return aco_objective_function(
        trial=trial,
        params=params,
        tuning_graphs=tuning_graphs,
        aco_class=ACOGraphColoring,
        verbose=ACO_VERBOSE
    )

print("Objective function wrapper ready!")

## 3. Run Hyperparmeters Tuning

In [ ]:
# Run the hyperparameter optimization
print(f"\nStarting hyperparameter optimization with {N_TRIALS} trials...")
print("=" * 70)

best_params = tuner.optimize(
    objective_func=objective_wrapper,
    param_config=PARAM_CONFIG,
    aco_class=ACOGraphColoring,
    n_trials=N_TRIALS
)

print("\n" + "=" * 70)
print("Optimization completed!")
print("\nBest Parameters Found:")
for param_name, param_value in best_params.items():
    print(f"  {param_name}: {param_value}")

In [ ]:
# Generate all optimization plots
print("\nGenerating optimization visualization plots...")
print("=" * 70)
tuner.generate_plots(recreate=True)
print("=" * 70)

### Best Trial Results

In [ ]:
# Display best trial graphs and metrics


# Get best trial number
best_trial_number = tuner.study.best_trial.number
best_trial_dir = Path(DATA_ROOT) / 'studies' / STUDY_NAME / 'results' / f'trial_{best_trial_number:04d}'

print(f"Best Trial: {best_trial_number}")
print(f"Best Objective Value: {tuner.study.best_value}")
print("=" * 70)

# Display metrics for best trial
display(Markdown(f"#### Best Trial Metrics"))

metric_files = [
    ('color_count.png', 'Color Count per Graph'),
    ('execution_time.png', 'Execution Time per Graph'),
    ('conflicts.png', 'Conflicts per Graph')
]

for filename, title in metric_files:
    metric_path = best_trial_dir / filename
    if metric_path.exists():
        display(Markdown(f"**{title}**"))
        display(HTML(f'<img src="{metric_path}" style="width: 75%; display: block; margin: 0 auto;">'))

# Display colored graphs for best trial
display(Markdown(f"#### Best Trial: Colored Graph Solutions"))

# Find all graph files in best trial directory
graph_files = sorted(best_trial_dir.glob('graph_*.png'))
for graph_file in graph_files:
    graph_name = graph_file.stem.replace('graph_', '')
    display(Markdown(f"**{graph_name}**"))
    display(HTML(f'<img src="{graph_file}" style="width: 75%; display: block; margin: 0 auto;">'))

### Display Study Visualizations

In [ ]:
# Display all study figures

study_figures_path = Path(DATA_ROOT) / 'studies' / STUDY_NAME / 'figures'

# List of all possible study figures with descriptions
figure_files = [
    ('history.png', 'Optimization History', 'Shows objective value progression across trials'),
    ('importances.png', 'Parameter Importances', 'Shows which hyperparameters impact results most'),
    ('slice.png', 'Slice Plots (All Parameters)', 'Shows how each parameter affects objective value'),
    ('timeline.png', 'Trial Timeline', 'Shows when each trial ran and how long it took')
]

print("Study Visualization Figures:")
print("=" * 70)

for filename, title, description in figure_files:
    figure_path = study_figures_path / filename
    if figure_path.exists():
        display(Markdown(f"### {title}"))
        display(Markdown(f"*{description}*"))
        display(HTML(f'<img src="{figure_path}" style="width: 75%; display: block; margin: 0 auto;">'))
        print(f"✓ {filename}")
    else:
        print(f"✗ {filename} (not generated - may require additional dependencies)")

print("=" * 70)

## 4. Comprehensive Algorithms Comparison

Compare three algorithms on the testing dataset:
- **Greedy Algorithm**: Fast constructive heuristic
- **Tabu Search**: Single-solution metaheuristic with memory
- **ACO**: Population-based metaheuristic (using best parameters from tuning)

Each algorithm runs 5 times on all test graphs to calculate statistics and generate comparison plots.

In [ ]:
# Import the comprehensive testing module
from comprehensive_testing import run_comprehensive_testing

# Set up testing directories - save comparison inside study path
comparison_output_dir = DATA_ROOT / 'studies' / STUDY_NAME / 'algorithm_comparison'

print(f"Data root: {DATA_ROOT}")
print(f"Dataset: {DATASET_NAME}")
print(f"Comparison output directory: {comparison_output_dir}")
print(f"  (Results will be saved in study: {STUDY_NAME})")

In [ ]:
# Prepare ACO parameters from best trial
aco_test_params = {
    'iterations': int(best_params['iterations']),
    'alpha': best_params['alpha'],
    'beta': best_params['beta'],
    'rho': best_params['rho'],
    'ant_count': int(best_params['ant_count']),
    'Q': best_params['Q'],
    'patience': best_params['patience']
}

print("\nACO Parameters for Testing (from best trial):")
for param, value in aco_test_params.items():
    print(f"  {param}: {value}")

In [ ]:
# Run comprehensive testing: 5 repetitions per algorithm per graph
# Results are cached in JSON - set force_rerun=True to re-execute tests
NUM_REPETITIONS = 5
FORCE_RERUN = False  # Set to True to ignore cached results and rerun testing

print(f"\n{'='*80}")
print(f"Running comprehensive testing with {NUM_REPETITIONS} repetitions per algorithm")
if FORCE_RERUN:
    print("⚠ FORCE_RERUN=True: Will ignore cached results and rerun all tests")
else:
    print("ℹ Using cached results if available (set FORCE_RERUN=True to rerun)")
print(f"{'='*80}\n")

df_comparison_results, df_comparison_stats = run_comprehensive_testing(
    data_root=str(DATA_ROOT),
    dataset_name=DATASET_NAME,
    output_dir=str(comparison_output_dir),
    num_repetitions=NUM_REPETITIONS,
    aco_params=aco_test_params,
    force_rerun=FORCE_RERUN
)

print("\n✓ Comprehensive testing complete!")

### 6.1 Display Statistics Summary

In [ ]:
# Display statistics table
from IPython.display import display, Markdown

print("Statistics Summary:")
display(df_comparison_stats)

# Print formatted statistics (without markdown)
print("\n" + "="*80)
print(df_comparison_stats.to_string(index=False))
print("="*80)

### 6.2 Display Comparison Visualizations

In [ ]:
# Display algorithm comparison plots (4 comparison metrics)
from IPython.display import Image

# Define all comparison plots
comparison_plots = [
    ('comparison_best_colors.png', 'Best Color Count Comparison (with Best Known Solutions)'),
    ('comparison_avg_colors.png', 'Average Color Count Comparison (with Std Dev)'),
    ('comparison_execution_time.png', 'Execution Time Comparison (log scale)'),
    ('comparison_conflicts.png', 'Conflict Count Comparison')
]

print("Algorithm Comparison Metrics:")
print("="*80)

for filename, title in comparison_plots:
    plot_path = comparison_output_dir / filename
    if plot_path.exists():
        display(Markdown(f"### {title}"))
        display(Markdown(f'<img src="{plot_path}" style="width: 75%; display: block; margin: 0 auto;"/>'))
    else:
        print(f"⚠ {title} not found: {filename}")

print("="*80)

### 6.3 Results Summary and File Locations

In [ ]:
# Print comprehensive testing results summary
print("="*80)
print("COMPREHENSIVE TESTING RESULTS SUMMARY")
print("="*80)

print(f"\n📁 Output Directory: {comparison_output_dir}")
print(f"   (Saved in study: {STUDY_NAME})")

print(f"\n📊 Generated Files:")
print(f"   - comparison_results.json (complete results - cached for reuse)")
print(f"   - comprehensive_test_results.csv (all {len(df_comparison_results)} individual runs)")
print(f"   - statistics_summary.csv (aggregated statistics)")
print(f"   - comparison_best_colors.png (best color count comparison)")
print(f"   - comparison_avg_colors.png (average color count with std dev)")
print(f"   - comparison_execution_time.png (execution time comparison)")
print(f"   - comparison_conflicts.png (conflict count comparison)")
print(f"   - color_distribution_boxplots.png (distribution analysis)")

print(f"\n💡 Tip: Results are cached in JSON. Re-running this cell will use cached data.")
print(f"   Set FORCE_RERUN=True to ignore cache and rerun all tests.")

print(f"\n🎯 Summary by Algorithm:")
for algo_name in df_comparison_stats['algorithm'].unique():
    df_algo = df_comparison_stats[df_comparison_stats['algorithm'] == algo_name]
    best_colors = df_algo['best_colors'].min()
    avg_colors = df_algo['avg_colors'].mean()
    avg_time = df_algo['avg_time'].mean()
    print(f"   {algo_name}:")
    print(f"      Best colors: {best_colors}")
    print(f"      Avg colors: {avg_colors:.2f}")
    print(f"      Avg time: {avg_time:.4f}s")

print("\n" + "="*80)